<a href="https://colab.research.google.com/github/Abuthalib/Abuthalib/blob/main/ResumeParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 8.2 MB/s eta 0:00:00

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [3]:
spacy.__version__

'3.7.2'

In [4]:
!nvidia-smi

Wed Nov 22 05:35:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [43]:
cv_data = json.load(open('/content/Data/train/train_data.json','r'))

In [44]:
len(cv_data)

200

In [45]:
!python -m spacy init fill-config /content/Data/train/base_config.cfg /content/Data/train/config.cfg

2023-11-22 08:22:16.724293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 08:22:16.724354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 08:22:16.724386: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 08:22:18.137985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/Data/train/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [46]:
len(cv_data)

200

In [47]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            # Checking for overlapping entities
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            entity_indices += list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = f"{[start, end]}    {text}\n"
                #print("Span", len(err_data))
                file.write(err_data)
            else:
                ents.append(span)
                #print("ents", len(ents))

        try:
            doc.ents = ents
            #print("doc", len(doc))
            db.add(doc)
            #print(len(db))
        except:
            pass

    return db

In [48]:
from sklearn.model_selection import train_test_split

In [49]:

train,test = train_test_split(cv_data,test_size=0.3)

In [50]:
len(train),len(test)

(140, 60)

In [51]:
file = open('error.txt','w' )

db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 94.70it/s]


In [52]:
len(db.tokens)

60

In [53]:
!python -m spacy train /content/Data/train/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-11-22 08:22:29.753847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 08:22:29.753900: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 08:22:29.753934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 08:22:31.271187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly i

In [54]:
nlp = spacy.load('/content/output/model-best')

In [55]:
doc = nlp('My name is Abu thalib. worked at Apple. i have 6 year of experince')

for ent in doc.ents:
  print(ent.text, "->>>>>>>>>>",ent.label_)

Abu thalib ->>>>>>>>>> Email Address
Apple ->>>>>>>>>> Companies worked at
6 year ->>>>>>>>>> Years of Experience


In [56]:
!pip install PyMuPDF

In [57]:
import sys,fitz

In [58]:
fname = '/content/Data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

if doc is None:
    print("Error: Could not open the document.")
else:
    text = " "
    for page in doc:
        text = text + str(page.get_text())

In [59]:
text = text.strip()

In [60]:
text = ' '.join(text.split())

In [61]:
text

'Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edg

In [62]:
doc = nlp(text)

for ent in doc.ents:
  print(ent.text, "->>>>>>>>>>",ent.label_)

Alice Clark ->>>>>>>>>> Name
Delhi ->>>>>>>>>> Location
Microsoft ->>>>>>>>>> Companies worked at
Software Engineer ->>>>>>>>>> Designation
Microsoft ->>>>>>>>>> Companies worked at
Microsoft ->>>>>>>>>> Companies worked at
Microsoft ->>>>>>>>>> Companies worked at
Microsoft ->>>>>>>>>> Companies worked at
Microsoft ->>>>>>>>>> Companies worked at
Microsoft ->>>>>>>>>> Companies worked at
Microsoft ->>>>>>>>>> Companies worked at
Indian Institute of Technology ->>>>>>>>>> College Name
Machine Learning, Natural Language Processing, and Big Data Handling ADDITIONAL INFORMATION Professional Skills • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal skills with ability to interact with individuals at all the levels • Quick learner and maintains cordial relationship with project manager and team members and good performer both in team and independent job environments • Positive attitude towards superiors &amp; peers • Supervised junior developers thr